# Analisando o bot

Este jupyter-notebook vai auxiliar na análise de detecção de intenções da Taís.

## Instalação

### Configurando jupyter

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

### Listando versões

In [2]:
import rasa_nlu

print("rasa_nlu: {}".format(rasa_nlu.__version__))

rasa_nlu: 0.14.1


### Treinando o modelo do Rasa NLU

Na celula abaixo todas as intents na pasta `../data/intents/` serão utilizadas para gerar a matrix de confuzão para de avaliação do bot.

Ela irá treinar o modelo e executar a avaliação que pode ser verificada na saída da célula abaixo.

In [3]:
import os

from rasa_nlu.training_data import load_data
from rasa_nlu.model import Trainer
from rasa_nlu import config


intents_directory = '../../bot/data/intents/'

intents = {}

for intent_file in os.listdir(intents_directory):
    intent_file_path = os.path.join(intents_directory, intent_file)

    intents[intent_file] = {}

    intent_list = []
    intent_name = None

    with open(intent_file_path) as f:
        lines = f.readlines()

        for line in lines:
            line = line.strip()

            if line.startswith('##'):
                if intent_name is not None:
                    intents[intent_file][intent_name] = intent_list
                intent_name = line.replace('## intent:', '') 
                intent_list = []

            elif line.startswith('- '):
                intent_list.append(line.replace('- ', ''))


# loading the nlu training samples
training_data = load_data(intents_directory)

# trainer to educate our pipeline
trainer = Trainer(config.load("../../bot/nlu_config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/captacao.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 43 (3 distinct intents)
	- Found intents: 'captacao_quando_captar', 'captacao_como_captar', 'captacao'
	- entity examples: 0 (0 distinct entities)
	- found entities: 

INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/salic.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 144 (14 distinct intents)
	- Found intents: 'salic_preenchimento_campo_custo_auditoria', 'salic_cadastro_proponente', 'salic_cadastro_usuario', 'salic_erros_vinculo_cpf_cnpj', 'salic_erros', 'salic_preenchimento_cadastro_agencia_bancaria', 'salic_erros_salvamento_de_proposta', 'salic_preenchimento_valor_ingresso', 'salic_preenchimento_vinculo_cpf_proposta', 'salic_preenchimento', 'salic_erros_achar_proposta', 'salic_preenchimento_cadastro_rubrica_advogado_conta

In [4]:
from rasa_nlu.evaluate import run_evaluation
run_evaluation('../../bot/data/intents/', model_directory)

INFO:tensorflow:Restoring parameters from /work/notebooks/intents/./models/nlu/default/current/intent_classifier_tensorflow_embedding.ckpt
INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/captacao.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 43 (3 distinct intents)
	- Found intents: 'captacao_quando_captar', 'captacao_como_captar', 'captacao'
	- entity examples: 0 (0 distinct entities)
	- found entities: 

INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/salic.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 144 (14 distinct intents)
	- Found intents: 'salic_preenchimento_campo_custo_auditoria', 'salic_cadastro_proponente', 'salic_cadastro_usuario', 'salic_erros_vinculo_cpf_cnpj', 'salic_erros', 'salic_preenchimento_cadastro_agencia_bancaria', 'salic_erros_salvamento_de_proposta', 'salic_preenchimento_valor_ingresso', 'salic_pre

INFO:rasa_nlu.evaluate:Your model made no errors
INFO:rasa_nlu.evaluate:Entity evaluation results:
INFO:rasa_nlu.evaluate:Evaluation for entity extractor: ner_crf 
INFO:rasa_nlu.evaluate:F1-Score:  0.9992427994011875
INFO:rasa_nlu.evaluate:Precision: 0.9992668923649722
INFO:rasa_nlu.evaluate:Accuracy:  0.9992663242846662
INFO:rasa_nlu.evaluate:Classification report: 
                   precision    recall  f1-score   support

      lei_rouanet       1.00      0.88      0.94        33
        no_entity       1.00      1.00      1.00      5162
situacao_processo       1.00      1.00      1.00       257

        micro avg       1.00      1.00      1.00      5452
        macro avg       1.00      0.96      0.98      5452
     weighted avg       1.00      1.00      1.00      5452



{'intent_evaluation': {'predictions': [{'text': 'quero falar sobre captacao',
    'intent': 'captacao',
    'predicted': 'captacao',
    'confidence': 0.9459694623947144},
   {'text': 'tenho duvidas sobre captacao',
    'intent': 'captacao',
    'predicted': 'captacao',
    'confidence': 0.9256139397621155},
   {'text': 'você sabe falar sobre captacao',
    'intent': 'captacao',
    'predicted': 'captacao',
    'confidence': 0.8973541259765625},
   {'text': 'quando poderei efetuar a captação de recursos do meu projeto',
    'intent': 'captacao_quando_captar',
    'predicted': 'captacao_quando_captar',
    'confidence': 0.9565302133560181},
   {'text': 'captação de recursos, quando fazer',
    'intent': 'captacao_quando_captar',
    'predicted': 'captacao_quando_captar',
    'confidence': 0.9672774076461792},
   {'text': 'qual é o momento para fazer a captação de recursos',
    'intent': 'captacao_quando_captar',
    'predicted': 'captacao_quando_captar',
    'confidence': 0.94622856378

### Lista de Problemas

O arquivo `erros,json` mostra os erros encontrados após executar o comando `rasa_nlu.evaluate.run_evaluation`.

Normalmente os erros mostrados são textos repetidos nos exemplos de diferrentes `intents`.

Caso o arquivo não seja gerado significa que não foram encontrados erros.

In [5]:
%cat errors.json

cat: errors.json: No such file or directory


### Avaliação do modelo de NLU treinado

Caso queira testar mensagens específicas mande a mensagem na célula seguinte:

In [6]:
pprint(interpreter.parse('posso terceirizar a captação de recursos?'))

{
  "intent": {
    "name": "captacao_quando_captar",
    "confidence": 0.7608696222305298
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "captacao_quando_captar",
      "confidence": 0.7608696222305298
    },
    {
      "name": "captacao_como_captar",
      "confidence": 0.6227313280105591
    },
    {
      "name": "elogios",
      "confidence": 0.36979344487190247
    },
    {
      "name": "processo_prazo",
      "confidence": 0.30230188369750977
    },
    {
      "name": "captacao",
      "confidence": 0.2410399466753006
    },
    {
      "name": "tudo_bem",
      "confidence": 0.2220241129398346
    },
    {
      "name": "processo_admissibilidade",
      "confidence": 0.2090226113796234
    },
    {
      "name": "despedir",
      "confidence": 0.20639105141162872
    },
    {
      "name": "lei_rouanet_analise_pela_cnic",
      "confidence": 0.20043903589248657
    },
    {
      "name": "lei_rouanet_divulgacao_patrocinio",
      "confidence": 0.19034498929